In [1]:
import pandas as pd
import re
import openpyxl 
import numpy as np
from timeit import default_timer as timer

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
#predefine parameters
#bank statement excel, worksheet names and trans description to be filtered out
bsExcelFile = "Reconciliation_20200716.xlsx"
bsExcelWsName = '07.16 3207971832Statement'
filterTransDesc = "HANDLING CHRG"

#payout worksheet names, excel, merchant worksheet name, 
payoutWsName = ['09.07-EU ',
                 '13.07-COP ',
                 '13.07-MANAGE COP ']
payoutExcelFile = "Raw/PAYOUT - 16.07.2020.xlsx"
merchantWsName = '13.07-MERCHANT '
billWsName = '14.07-BILL PAYMENT '

#name col to remove rows with missing na or to match bs
payoutNameCol = 'Member Name'
merchantNameCol = 'Merchant'
merchantBankNameCol = 'Bank Acc Name'
billPaymentNameCol = "Name"

#col of payout to match amount (payout, merchant and bill payment must have)
poAmountCol = "Amount"
#col of final output as name
poNameCol = "Member Name"
bsName = "BS Name"

#cols to match in bank statement
bsColMatchList = ['Reference 1', 'Reference 2', 'Reference 3', 'Reference 4']
bsAmountCol = "Debit Amount"

#index col to track rows
payoutIndex = "PO_Index"
merchantIndex = "Merchant_Index"
billPaymentIndex = "BP_Index"
bsIndex = "BS_Index"

In [4]:
#reconciled payout
excelFile = "Reconciliation_20200716.xlsx"
wsName = "Reconciled"
dfReconciled = pd.read_excel(excelFile, sheet_name = wsName)

#not reconciled payout
excelFile = "Reconciliation_20200716.xlsx"
wsName = "Unreconciled_Payout"
dfNotReconciledPO = pd.read_excel(excelFile, sheet_name = wsName)

#not reconciled bs
excelFile = "Reconciliation_20200716.xlsx"
wsName = "Unreconciled_BS"
dfNotReconciledBS = pd.read_excel(excelFile, sheet_name = wsName)

In [29]:
dfReconciled.head()

,Withdraw Date,Username,Member Name,Withdraw,Package,Amount,Unnamed: 6,Unnamed: 7,Remarks,BANK STATEMENT,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,2020-07-13,Loh Chong Chun,Loh Chong Chun,NaN,NaN,0.58,Edit,Reject,Merchant_20200713,0.58,3207971832PM16072005 MERCHANT REDEEM,MERCHANT REDEEM 13072020,Loh Chong Chun
1,2020-07-13,MW MODERN WORLD,MW MODERN WORLD,NaN,NaN,1.00,Edit,Reject,Merchant_20200713,1.00,3207971832PM16072005 MERCHANT REDEEM,MERCHANT REDEEM 13072020,MW MODERN WORLD
2,2020-07-13,KAMUNTING VEGETARIAN,NGA YEIT HEE,NaN,NaN,1.56,Edit,Reject,Merchant_20200713,1.56,3207971832PM16072009 MERCHANT REDEEM,MERCHANT REDEEM 13072020,NGA YEIT HEE
3,2020-07-09,CELINESUN,SUN SIEW HOOI,NaN,EU,2.00,Edit,Reject,EU_20200709,2.00,3207971832PM16072055 EU REFUND,EU REFUND 9072020,SUN SIEW HOOI
4,2020-07-13,7 CIRCLE SDN BHD,7 CIRCLE SDN BHD,NaN,NaN,3.24,Edit,Reject,Merchant_20200713,3.24,3207971832PM16072011 MERCHANT REDEEM,MERCHANT REDEEM 13072020,7 CIRCLE SDN BHD


### Read File

In [3]:
#read bank statement file
dfBankStatement = pd.read_excel(bsExcelFile, sheet_name = bsExcelWsName)

#transform bank statement 
content = dfBankStatement.loc[18:,]
dfBS = pd.DataFrame(content)
dfBS.columns = dfBankStatement.loc[16,].values
dfBS = dfBS.query('`Transaction Description`!= "%s"'%filterTransDesc)

#combine ref columns
# colsToCombine = ['Reference 1', 'Reference 2', 'Reference 3', 'Reference 4']
# dfBS['Combined'] = dfBS[colsToCombine].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

#read payout files
pandaList = []
for sheetName in payoutWsName:
    dfExcel = pd.read_excel(payoutExcelFile, sheet_name = sheetName)
    pandaList.append(dfExcel)
dfPayOut = pd.concat(pandaList, axis = 0).copy()
#remove empty member name
dfPayOut = dfPayOut[pd.notnull(dfPayOut[payoutNameCol])]

#merchant file
dfMerchant = pd.read_excel(payoutExcelFile, sheet_name = merchantWsName)
#remove empty merchant col
dfMerchant = dfMerchant[pd.notnull(dfMerchant[merchantNameCol])]


dfBillPayment = pd.read_excel(payoutExcelFile, sheet_name = billWsName)
#remove empty name
dfBillPayment = dfBillPayment[pd.notnull(dfBillPayment[billPaymentNameCol])]

#create index
dfPayOut[payoutIndex] = np.arange(dfPayOut.shape[0])
dfMerchant[merchantIndex] = np.arange(dfMerchant.shape[0])
dfBillPayment[billPaymentIndex] = np.arange(dfBillPayment.shape[0])
dfBS[bsIndex] = np.arange(dfBS.shape[0])

colList = [bsIndex] + bsColMatchList + [bsAmountCol]
dfBSFinal = dfBS[colList].copy()

#ruleDict, which file type to use which set of rules
ruleDict = {"PayOut":[1, 2, 3, 4, 6],
           "Merchant":[1, 2, 3, 4],
           "BillPayment":[5]}

NameError: name 'excelFile' is not defined

In [38]:
def exactMatch(df, indexName, nameCol):
    startTime = timer()
    mergeList = []
    indexList = list(df[pd.notnull(df[nameCol])][indexName].values)
    for col in bsColMatchList:
        dfTemp = df[df[indexName].isin(indexList)].copy()
        dfMerged = dfTemp.merge(dfBSFinal, how = 'inner', left_on = [nameCol, poAmountCol],
                                                            right_on = [col, bsAmountCol])
        dfMerged2 = dfMerged[(pd.notnull(dfMerged[nameCol])) & (pd.notnull(dfMerged[col]))].copy()
        dfMerged2.rename(columns = {col: bsName, nameCol : poNameCol}, inplace = True)
        mergeList.append(dfMerged2)
        #update index list

    
#     dfMergedFinal = pd.concat(mergeList).copy()[[indexName, poNameCol, poAmountCol, 
#                                                  bsIndex, bsName, bsAmountCol]]
    dfMergedFinal = pd.concat(mergeList).copy()
    dfMergedFinal['Category'] = "ExactMatch"
    endTime = timer()
    print("Total time: %0.4fs" % (endTime - startTime))
    return dfMergedFinal

In [39]:
#exact matching for payout file
dfExactMatchPO = exactMatch(dfPayOut, payoutIndex, payoutNameCol)
#find no match list
noMatchPOList = set(dfPayOut[payoutIndex].values)\
                        .difference(set(dfExactMatchPO[payoutIndex].values))

#exact matching for bill payment file
dfExactMatchBP = exactMatch(dfBillPayment, billPaymentIndex, billPaymentNameCol)
#find no match list
noMatchBPList = set(dfBillPayment[billPaymentIndex].values)\
                        .difference(set(dfExactMatchBP[billPaymentIndex].values))

#exact matching for merchant file
exactMatchMerchantList = []
for colName in [merchantNameCol, merchantBankNameCol]:
    dfExactMatchMerchant = exactMatch(dfMerchant, merchantIndex, colName)
#     dfExactMatchMerchant.rename(columns = {colName: 'Member Name'}, inplace = True)
    exactMatchMerchantList.append(dfExactMatchMerchant)

dfExactMatchMerchant = pd.concat(exactMatchMerchantList).copy()
dfExactMatchMerchant.drop_duplicates(subset=[merchantIndex, bsIndex], inplace = True)

#harmonize no match list
noMatchMerchantList = set(dfMerchant[merchantIndex].values)\
                        .difference(set(dfExactMatchMerchant[merchantIndex].values))

#combine all exact matches
dfExactMatchFinal = pd.concat([dfExactMatchPO, dfExactMatchBP, dfExactMatchMerchant]).copy()
matchedBSIndex = dfExactMatchFinal[bsIndex].values
dfExactMatchFinal.shape

Total time: 0.0759s
Total time: 0.0826s
Total time: 0.0998s
Total time: 0.1259s


(648, 40)

In [40]:
dfExactMatchFinal.columns

Index(['Country', 'Withdraw Date', 'Username', 'Member Name', 'Withdraw',
       'Package', 'Bank', 'Bank Acc', 'Bank-in Slip', 'Bank-in By',
       'Bank-in Date', 'Remarks', 'Status', 'Amount', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 13', 'PO_Index', 'BS_Index', 'BS Name',
       'Reference 2', 'Reference 3', 'Reference 4', 'Debit Amount',
       'Reference 1', 'Category', 'Bill Date', 'Bill Type', 'Acc Holder',
       'Acc Number', 'Reference', 'Account ID', 'Mobile Phone', 'Action',
       'BP_Index', 'Bank Acc Name', 'Unnamed: 11', 'Unnamed: 12',
       'Merchant_Index', 'Merchant'],
      dtype='object')

In [67]:
csvFile = "2021-01-21_ExactMatchAllCols.csv"
dfExactMatchFinal.to_csv(csvFile, index = False)

In [18]:
#get unmatched list PO for partial matching purpose
noMatchPOlist = set(dfPayOut[payoutIndex].values).difference(set(dfExactMatchPO[payoutIndex].values))
len(noMatchPOlist),len(dfPayOut[payoutIndex].values), len(dfExactMatchPO[payoutIndex].values)

(425, 780, 355)

In [19]:
#get unmatched list BP for partial matching purpose
noMatchBPlist = set(dfBillPayment[billPaymentIndex].values).difference(set(dfExactMatchBP[billPaymentIndex].values))
len(noMatchBPlist)

237

In [20]:
#get unmatched list merchant for partial matching purpose
noMatchMerchantlist = set(dfMerchant[merchantIndex].values).difference(set(dfExactMatchMerchant[merchantIndex].values))
len(noMatchMerchantlist)

227

### Partial match

In [14]:
def matchFirst2Terms(name, series):
    ruleName = "matchFirst2Terms"
    #match series with first 1 or 2 terms of names
    if len(name.split(' ')) >= 3:
        match = ' '.join(name.split(' ')[:2])
        resultArray = series.map(lambda x:ruleName if re.search('%s'%match, str(x), flags = re.IGNORECASE)
                                else "False").values
    elif len(name.split(' ')) == 2:
        match = name.split(' ')[0]
        resultArray = series.map(lambda x:ruleName if re.search('%s'%match, re.sub(" ","",str(x)), flags = re.IGNORECASE)
                                else "False").values
    else:
        resultArray = np.full(len(series), "False")

    return resultArray

def matchTruncated(name, series):
    ruleName = "matchTruncated"
    #match series with truncated name if there is 
    rule1 =  series\
            .map(lambda x: True if re.search('%s'%re.escape(str(name)), str(x), flags = re.IGNORECASE) 
                 and str(x) != 'nan' else False).values
    #match name with truncated name in the series if there is
    rule2 = series\
                .map(lambda x: True if re.search('%s'%re.escape(str(x)), str(name), flags = re.IGNORECASE) 
                     and str(x) != 'nan' else False).values
    resultArray = (rule1|rule2)
    resultArray = np.where(resultArray, ruleName, "False")
    
    return resultArray

def noSpacenoCopRefund(name, series):
    ruleName = "noSpacenoCopRefund"
    #remove cop refund and remove space for BS reference to match wth name
    rule1 = series.map(lambda x:True if re.search('%s'%re.escape(str(name)), 
                            re.sub(' ','',re.sub("COP REFUND", '',str(x))), flags = re.IGNORECASE)
                                else False).values
    rule2 = series.map(lambda x:True if re.search('%s'%re.escape(re.sub(' ','',re.sub("COP REFUND", '',str(x)))), 
                        str(name), flags = re.IGNORECASE) and re.sub(' ','',re.sub("COP REFUND", '',str(x)))!=''
                            else False).values
    resultArray = (rule1|rule2)
    resultArray = np.where(resultArray, ruleName, "False")
    return resultArray

def matchAccountLast4Digit(accountNo, series):
    ruleName = "matchAccountLast4Digit"
    resultArray = series.map(lambda x:ruleName if re.search('%s'%accountNo, str(x), flags = re.IGNORECASE)
                                        and len(accountNo) == 4 else "False").values
    return resultArray

def matchBillTypeRef(billType, series):
    ruleName = "matchBillTypeRef"
    if billType == '':
        resultArray = np.full(len(series), "False")
        reason = ""
    else:
        #if can find billtype - Indah water in reference 
        rule1 =  series\
                .map(lambda x: True if re.search('%s'%billType, str(x), flags = re.IGNORECASE) 
                     and str(x) != 'nan' else False).values
        #if billtype is PAIP and Pengurusan air pahan in reference
        rule2 = series\
                .map(lambda x: True if re.search('%s'%"PENGURUSAN AIR PAHAN", str(x), flags = re.IGNORECASE) 
                     and billType == 'PAIP' else False).values
        #if billtype is PBA and perbadanan bekalan air in reference
        rule3 = series\
                .map(lambda x: True if re.search('%s'%"perbadanan bekalan a", str(x), flags = re.IGNORECASE) 
                     and billType == 'PBA' else False).values
        resultArray = (rule1|rule2|rule3)

        resultArray = np.where(resultArray, ruleName, "False")
    return resultArray

def matchUniqueAmount(amount, series):
    ruleName = "matchUniqueAmount"
    #if only unique match in bs for amount
    resultArray = series.map(lambda x:True if x == amount else False).values
    if sum(resultArray) == 1:
        return np.full(len(series), ruleName)
    else:
        return np.full(len(series), "False")

In [15]:
def partialMatch(df, notMatchIndex, indexCol, nameCol, ruleType):
    matchResult = []
    startTime = timer()
    for index in notMatchIndex:
        #instantiate rule
        dfSubset = df.query('%s == %s'%(indexCol, index))
        if len(dfSubset[nameCol].values) != 0:
            POname = dfSubset[nameCol].values[0]
            POname = re.sub("\u200b", "", POname)
            POamount = dfSubset[poAmountCol].values[0]
            #merchant, other payout
            if "Bank Acc" in dfSubset.columns:
                AccountNo = re.sub(' |-|\.0','', str(dfSubset['Bank Acc'].values[0]))[-4:]
            #bill payment
            elif "Acc Number" in dfSubset.columns:
                AccountNo = re.sub(' |-|\.0','', str(dfSubset['Acc Number'].values[0]))[-4:]
            else:
                AccountNo = ''
            if 'Bill Type' in dfSubset.columns:
                BillType = dfSubset['Bill Type'].values[0]
            else:
                BillType = ''
            for col in ['Reference 1', 'Reference 2', 'Reference 3', 'Reference 4']:
                match1 = matchTruncated(POname, dfBSFinal[col])
                match2 = matchFirst2Terms(POname, dfBSFinal[col])
                match3 = noSpacenoCopRefund(POname, dfBSFinal[col])
                match4 = matchAccountLast4Digit(AccountNo, dfBSFinal[col])
                match5 = matchBillTypeRef(BillType, dfBSFinal[col])
                match6 = matchUniqueAmount(POamount, dfBSFinal[bsAmountCol])
                ruleStr1 = "|".join(['(match%s!="False")'%i for i in ruleDict[ruleType]])
                checkRule = eval(ruleStr1)
                #assign bool statement to checkRule
                dfMatch = dfBSFinal[(dfBSFinal[bsAmountCol] == POamount) & (checkRule)]
                ruleStr1 = '[' + ','.join(['match%s'%i for i in ruleDict[ruleType]]) + ']'
                #assign list to ruleList
                ruleList = eval(ruleStr1)
                if dfMatch.shape[0] == 1:
                    BSname = dfMatch[col].values[0]
                    BSamount = dfMatch[bsAmountCol].values[0]   
                    BSindex = dfMatch[bsIndex].values[0]
                    rule = ','.join([i[BSindex] for i in ruleList if i[BSindex]!="False"])
                    matchResult.append([index, POname, POamount, BSindex, BSname, BSamount, rule])
    #             more than 1 match
                elif dfMatch.shape[0] > 1:
                    for no, name in enumerate(dfMatch[col].values):
                        BSname = name
                        BSamount = dfMatch[bsAmountCol].values[no]
                        BSindex = dfMatch[bsIndex].values[no]
                        rule = ','.join([i[BSindex] for i in ruleList if i[BSindex]!="False"])
                        #nid to remove duplicate after consolidation
                        matchResult.append([index, POname, POamount, BSindex, BSname, BSamount, rule])

    endTime = timer()
    print("Total time: %0.4fs" % (endTime - startTime))
    return matchResult

In [16]:
def constructTable(matchResult, df, indexName, nameCol):
    pandaList = []
    for i in matchResult:
        pandaList.append(i)
    
    dfMatchFinal = pd.DataFrame(pandaList)
    if dfMatchFinal.shape[0]!=0:
        #[index, POname, POamount, BSindex, BSname, BSamount, rule]
        dfMatchFinal.columns = [indexName, poNameCol, poAmountCol, bsIndex, bsName, bsAmountCol, 'Reason']
        dfMatchFinal['Category'] = "PartialMatch"
        return dfMatchFinal
    else:
        return None

In [21]:
matchResultPO = partialMatch(dfPayOut, noMatchPOlist, 
                             payoutIndex, payoutNameCol, ruleType = "PayOut")

dfPartialMatchPO = constructTable(matchResultPO, dfPayOut, payoutIndex, poNameCol)

dfPartialMatchPO.drop_duplicates(payoutIndex, inplace = True)

dfPartialMatchPO.shape[0]

Total time: 321.0851s


416

In [22]:
matchResultBP = partialMatch(dfBillPayment, noMatchBPlist, 
                             billPaymentIndex, billPaymentNameCol, ruleType = "BillPayment")

dfPartialMatchBP = constructTable(matchResultBP, dfBillPayment, billPaymentIndex, billPaymentNameCol)
#ll only keep d first record of multiple matches
dfPartialMatchBP.drop_duplicates(billPaymentIndex, inplace = True)

dfPartialMatchBP.shape[0]

Total time: 199.0323s


24

In [23]:
MerchantList = []
for colName in ['Merchant', 'Bank Acc Name']:
    matchResultMerchant = partialMatch(dfMerchant, noMatchMerchantList, merchantIndex, colName, ruleType = "Merchant")
    dfPartialMatchMerchant = constructTable(matchResultMerchant, dfMerchant, merchantIndex, colName)
    MerchantList.append(dfPartialMatchMerchant)
    
dfPartialMatchMerchant = pd.concat(MerchantList).copy()
dfPartialMatchMerchant.drop_duplicates(subset=['Merchant_Index', 'BS_Index'], inplace = True)
dfPartialMatchMerchant.shape[0]

Total time: 168.1403s
Total time: 173.4157s


226

In [25]:
#combine all partial matches
dfPartialMatchFinal = pd.concat([dfPartialMatchPO, dfPartialMatchBP, dfPartialMatchMerchant]).copy()
dfPartialMatchFinal.shape

(666, 10)

In [49]:
dfPartialMatchPO.head()

,PO_Index,Member Name,Amount,BS_Index,BS Name,Debit Amount,Reason,Category
0,1,KHAIRUL AZIZUL BIN ANANG,140.0,231,KHAIRUL AZIZUL B,140,"matchTruncated,matchFirst2Terms",PartialMatch
3,7,md yasin bin mohd salleh,600.0,508,md yasin bin moh,600,"matchTruncated,matchFirst2Terms",PartialMatch
4,10,REZA AL-HAFIZ BIN ABDUL RAHIM,400.0,390,REZA AL-HAFIZ BI,400,"matchTruncated,matchFirst2Terms",PartialMatch
5,12,Muhammad wafi bin husain,50.0,110,Muhammad wafi bi,50,"matchTruncated,matchFirst2Terms",PartialMatch
6,26,Zalina binti Zainal Abidin,982.0,650,3207971832PM16072053 EU REFUND,982,matchUniqueAmount,PartialMatch


In [55]:
colList = set(dfPayOut.columns).difference(set(dfPartialMatchPO.columns))
colList = list(colList) + ['PO_Index']
dfPartialMatchPO2 = dfPartialMatchPO.merge(dfPayOut[colList], on = 'PO_Index')

In [59]:
dfPartialMatchPO2.shape

(416, 23)

In [58]:
colList = set(dfBillPayment.columns).difference(set(dfPartialMatchBP.columns))
colList = list(colList) + ['BP_Index']
dfPartialMatchBP2 = dfPartialMatchBP.merge(dfBillPayment[colList], on = 'BP_Index')

In [60]:
dfPartialMatchBP2.shape

(24, 18)

In [62]:
colList = set(dfMerchant.columns).difference(set(dfPartialMatchMerchant.columns))
colList = list(colList) + ['Merchant_Index']
dfPartialMatchMerchant2 = dfPartialMatchMerchant.merge(dfMerchant[colList], on = 'Merchant_Index')

In [63]:
dfPartialMatchMerchant2.shape

(226, 20)

In [64]:
dfPartialMatchFinal2 = pd.concat([dfPartialMatchPO2, dfPartialMatchBP2, dfPartialMatchMerchant2]).copy()
dfPartialMatchFinal2.shape

(666, 38)

In [66]:
dfPartialMatchFinal3 = dfPartialMatchFinal2.merge(dfBS, on = "BS_Index")

In [69]:
dfPartialMatchFinal3.head()

,PO_Index,Member Name,Amount,BS_Index,BS Name,Debit Amount_x,Reason,Category,Bank,Country,Bank-in Slip,Bank-in By,Unnamed: 15,Unnamed: 13,Package,Withdraw,Status,Username,Withdraw Date,Bank-in Date,Bank Acc,Unnamed: 14,Remarks_x,BP_Index,Reference,Acc Holder,Action,Bill Type,Account ID,Name,Mobile Phone,Acc Number,Bill Date,Merchant_Index,Unnamed: 12,Merchant,Unnamed: 11,Bank Acc Name,Trn. Date,Cheque No/Ref No,Transaction Description,Debit Amount_y,Credit Amount,Reference 1,Reference 2,Reference 3,Reference 4,Reference 5,Reference 6,Remarks_y
0,1.0,KHAIRUL AZIZUL BIN ANANG,140.0,231,KHAIRUL AZIZUL B,140.0,"matchTruncated,matchFirst2Terms",PartialMatch,MAYBANK BERHAD,MALAYSIA,NaN,NaN,Reject,NaN,EU,NaN,Pending,khairulpay,2020-07-09 00:00:00,NaN,163091087216,Edit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,16/07/2020,000053,MISC DR,140,NaN,3207971832PM16072053 EU REFUND,EU REFUND 9072020,KHAIRUL AZIZUL B,NaN,NaN,NaN,EU
1,7.0,md yasin bin mohd salleh,600.0,508,md yasin bin moh,600.0,"matchTruncated,matchFirst2Terms",PartialMatch,MAYBANK BERHAD,MALAYSIA,NaN,NaN,Reject,NaN,EU,NaN,Pending,yasinsalleh,2020-07-09 00:00:00,NaN,163082097270,Edit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,16/07/2020,000041,MISC DR,600,NaN,3207971832PM16072053 EU REFUND,EU REFUND 9072020,md yasin bin moh,NaN,NaN,NaN,EU
2,10.0,REZA AL-HAFIZ BIN ABDUL RAHIM,400.0,390,REZA AL-HAFIZ BI,400.0,"matchTruncated,matchFirst2Terms",PartialMatch,MAYBANK BERHAD,MALAYSIA,NaN,NaN,Reject,NaN,EU,NaN,Pending,MEDINAMIKAYLA,2020-07-09 00:00:00,NaN,163037039848,Edit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,16/07/2020,000039,MISC DR,400,NaN,3207971832PM16072053 EU REFUND,EU REFUND 9072020,REZA AL-HAFIZ BI,NaN,NaN,NaN,EU
3,12.0,Muhammad wafi bin husain,50.0,110,Muhammad wafi bi,50.0,"matchTruncated,matchFirst2Terms",PartialMatch,CIMB-MALAYSIA,MALAYSIA,NaN,NaN,Reject,NaN,EU,NaN,Pending,Wafi,2020-07-09 00:00:00,NaN,7051323340,Edit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,16/07/2020,000035,MISC DR,50,NaN,3207971832PM16072053 EU REFUND,EU REFUND 9072020,Muhammad wafi bi,NaN,NaN,NaN,EU
4,26.0,Zalina binti Zainal Abidin,982.0,650,3207971832PM16072053 EU REFUND,982.0,matchUniqueAmount,PartialMatch,CIMB-MALAYSIA,MALAYSIA,NaN,NaN,Reject,NaN,EU,NaN,Pending,zalina78,2020-07-09 00:00:00,NaN,7048684633,Edit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,16/07/2020,000019,MISC DR,982,NaN,3207971832PM16072053 EU REFUND,EU REFUND 9072020,Zalina binti Zai,NaN,NaN,NaN,EU


In [68]:
csvFile = "2021-01-21_PartialMatchAllCols.csv"
dfPartialMatchFinal3.to_csv(csvFile, index = False)